In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline
from sklearn.linear_model.logistic import LogisticRegression

def gaussModel(vitals_train,labs_train,age_train):

    # Merging vitals_train & labs_train
    vitals_labs = pd.merge(vitals_train,labs_train, left_index=True,right_index=True, how='outer')
    
    vitals_labs_icu = vitals_labs[vitals_labs['ICU'] == 1]

    vitals_labs2 = vitals_labs_icu.drop(['ID_x', 'TIME_x', 'ICU', 'ID_y', 'TIME_y'], axis=1)
    vitals_labs2_mean = np.array(vitals_labs2.mean(axis=0))
    vitals_labs2_std = np.array(vitals_labs2.std(axis=0))

    thres1 = (vitals_labs2_mean - 2*vitals_labs2_std)
    thres2 = (vitals_labs2_mean + 2*vitals_labs2_std)
    thres1_labels = pd.DataFrame(thres1,vitals_labs2.columns)
    thres2_labels = pd.DataFrame(thres2,vitals_labs2.columns)

    for col in vitals_labs2.columns:
        for row in vitals_labs2.index:
            if vitals_labs2[col][row] > thres1_labels[0][col] and vitals_labs2[col][row] < thres2_labels[0][col]:
                vitals_labs2[col][row] = 0
            else:
                vitals_labs2[col][row] = 1
                
    vitals_labs_icu_ID = vitals_labs_icu.ix[:,['ID_x', 'TIME_x']]
    vitals_labs_icu_ID = pd.merge(vitals_labs_icu_ID, vitals_labs2, left_index=True,right_index=True, how='outer')
        
    return vitals_labs_icu_ID


data_path = os.path.abspath(os.path.dirname("a"))

vitals_train = pd.read_csv("%s/Training_Dataset/id_time_vitals_train.csv" % data_path)
labs_train = pd.read_csv("%s/Training_Dataset/id_time_labs_train.csv" % data_path)
age_train = pd.read_csv("%s/Training_Dataset/id_age_train.csv" % data_path)
label_train = pd.read_csv("%s/Training_Dataset/id_label_train.csv" % data_path)

train_gaussModel = gaussModel(vitals_train,labs_train,age_train)

train_gaussModel['LABEL'] = np.nan 
for dummy,id in list(enumerate(label_train.ID)):
    id_index = train_gaussModel.ID_x == id
    train_gaussModel.LABEL[id_index] = label_train.LABEL[dummy]
        

vitals_test = pd.read_csv("%s/Test_Dataset/id_time_vitals_test.csv" % data_path)
labs_test = pd.read_csv("%s/Test_Dataset/id_time_labs_test.csv" % data_path)
age_test = pd.read_csv("%s/Test_Dataset/id_age_test.csv" % data_path)

test_gaussModel = gaussModel(vitals_test,labs_test,age_test)


# create train feature
X_train = train_gaussModel.iloc[:,2:33]
y_train = train_gaussModel.iloc[:,33:]
# assert X_train.shape[0] == y_train.size
# fit model
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Test data
X_test = test_gaussModel.iloc[:,2:33]
# predict test data
predict_out = clf.predict(X_test)

# test_match = (predict_proba == np.array(label_valid['LABEL']))
# percentage = (float(np.sum(test_match))/float(np.size(predict_proba)))*100
# print percentage

df = test_gaussModel.iloc[:,:2]
df['prediction'] = predict_out


df.to_csv('output.csv',index=False)